In [1]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn import linear_model,tree,ensemble
from sklearn.metrics import accuracy_score,log_loss

import sys
sys.path.insert(0,"../libs")
import data_cleaning as dc

In [37]:
train = pd.read_csv("../data/train.csv")
train = train[train.AnimalType=="Dog"]
train = dc.massage_df(train)

In [78]:
akc = pd.read_json("../breeds/akc.json")
manual = [pd.Series({
    "breed":"American Pit Bull Terrier"
,"energy":"high","group":"Terrier Group"
,"rank":-1,"size":"medium"})
          ,pd.Series({
    "breed":"Miniature Poodle"
,"energy":"medium","group":"Toy Group"
,"rank":8,"size":"small"})]
akc = akc.append(manual,ignore_index=True)

In [79]:
akc["dog_breed"] = akc.breed.apply(lambda x:x.replace("Dog","").strip())

In [84]:
counts = train.akc_name.value_counts()
popular_breeds = counts[counts>100].index

In [91]:
# restrict training set by available data
train = train[ train.akc_name.apply(lambda x:x in akc.dog_breed.values) ]

In [107]:
akc_dict = akc[["dog_breed","energy","rank","size"]].set_index("dog_breed").to_dict()
for key in akc_dict.keys():
    train[key] = train.akc_name.apply(lambda x:akc_dict[key][x])
# end for

In [114]:
mytrain,mytest = train_test_split(train,test_size=0.4)

In [146]:
features = ["neuter_status","age_numeric_years","is_weekend"
            ,"energy","size","rank"]
# these features are linear combinations of the others
drop_features = ["neuter_status_Unknown"]

usable_mytrain_idx = mytrain[features].dropna().index
print "train utilization: %1.3f" % (float(len(usable_mytrain_idx))/len(mytrain))
mytrain = mytrain.ix[ usable_mytrain_idx ]
usable_mytest_idx = mytest[features].dropna().index
print "test utilization: %1.3f" % (float(len(usable_mytest_idx))/len(mytest))
mytest = mytest.ix[ usable_mytest_idx ]

mytrain_features = pd.get_dummies(mytrain[features])
mytrain_features = mytrain_features.drop(drop_features,axis=1)
mytrain_outcomes = mytrain.OutcomeType

mytest_features = pd.get_dummies(mytest[features])
mytest_features = mytest_features.drop(drop_features,axis=1)
mytest_outcomes = mytest.OutcomeType

train utilization: 1.000
test utilization: 1.000


In [147]:
# build classifier
#classifier = linear_model.LogisticRegression()
#classifier = tree.DecisionTreeClassifier(max_depth=3)
classifier = ensemble.RandomForestClassifier(max_depth=6)

# train
classifier.fit(mytrain_features,mytrain_outcomes);

In [148]:
predictions = classifier.predict(mytest_features)
prediction_probabilities = classifier.predict_proba(mytest_features)

# check performance
acc = accuracy_score(mytest_outcomes, predictions)
los = log_loss( pd.get_dummies( mytest.OutcomeType ).values, prediction_probabilities)
print("accuracy = {0:1.2f} \nlog-loss = {1:1.3f}".format(acc,los) )

accuracy = 0.56 
log-loss = 1.025


In [149]:
pd.Series(predictions).value_counts()

Adoption           3274
Return_to_owner    1420
Transfer            937
Euthanasia            3
dtype: int64

I predict lots of happy adoptions. In practice, there are many more transfers and euthanasia ...

In [150]:
mytest_outcomes.value_counts()

Adoption           2335
Return_to_owner    1571
Transfer           1422
Euthanasia          293
Died                 13
Name: OutcomeType, dtype: int64